In [6]:
from datasets import load_from_disk, load_metric
from transformers import RobertaTokenizer, RobertaConfig, RobertaForSequenceClassification, Trainer
import numpy as np
import pyarrow as pa
import pandas as pd

TypeError: Couldn't build proto file into descriptor pool: duplicate file name (sentencepiece_model.proto)

In [ ]:
# load
datadir = '/work3/s174498/sst2_dataset/'
#checkpoint = "/work3/s174498/finetuning-sentiment-model-all-samples-test5/checkpoint-13000"
checkpoint = "/work3/s174498/finetuning-sentiment-model-all-samples-test14/checkpoint-1500"

# test data
test_dataset = load_from_disk(datadir + 'test_dataset')

# tokenizer
tokenizer = RobertaTokenizer.from_pretrained(checkpoint)
tokenizer.model_max_len=512

# model
#config = RobertaConfig.from_pretrained(checkpoint, output_attentions=True, output_hidden_states=True)
config = RobertaConfig.from_pretrained(checkpoint)
config.output_hidden_states = True
#config.output_attentions= True
model = RobertaForSequenceClassification.from_pretrained(checkpoint, config=config)

In [ ]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True, max_length=512)

In [ ]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /work3/s174498/sst2_dataset/test_dataset/cache-d3396ff3c898c248.arrow


In [ ]:
tokenized_test

Dataset({
    features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1821
})

In [ ]:
trainer = Trainer(
    model=model,                        
    tokenizer=tokenizer
)

In [ ]:
# Predicting with model
predictions = trainer.predict(tokenized_test)


The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: sentence, idx. If sentence, idx are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1821
  Batch size = 8


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 33 but got size 36 for tensor number 1 in the list.

In [ ]:
#dataset_test_pred = list(np.argmax(predictions.predictions, axis=-1))
dataset_test_pred = predictions.label_ids

In [ ]:
predictions.predictions[0]

array([[ 2.0679846, -1.3902186],
       [-3.3308682,  2.781076 ],
       [ 2.5501764, -2.224367 ],
       ...,
       [-1.4489131,  1.2342678],
       [-3.3256083,  2.764728 ],
       [-3.1752741,  2.659728 ]], dtype=float32)

In [ ]:
predictions

PredictionOutput(predictions=(array([[ 2.0679846, -1.3902186],
       [-3.3308682,  2.781076 ],
       [ 2.5501764, -2.224367 ],
       ...,
       [-1.4489131,  1.2342678],
       [-3.3256083,  2.764728 ],
       [-3.1752741,  2.659728 ]], dtype=float32), (array([[[ 1.6696733e-01, -4.1883286e-02,  3.9459802e-03, ...,
         -7.5790197e-02,  8.4996082e-02,  1.1639428e-02],
        [ 9.6462548e-02,  1.7602514e-02, -2.5241684e-02, ...,
         -4.2261589e-01,  1.3482298e-01,  2.3019172e-01],
        [-2.2047957e-02,  1.2598801e-01,  5.7439271e-02, ...,
          1.4196399e-01,  3.6889020e-02, -4.8910141e-02],
        ...,
        [-1.0000000e+02, -1.0000000e+02, -1.0000000e+02, ...,
         -1.0000000e+02, -1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02, -1.0000000e+02, ...,
         -1.0000000e+02, -1.0000000e+02, -1.0000000e+02],
        [-1.0000000e+02, -1.0000000e+02, -1.0000000e+02, ...,
         -1.0000000e+02, -1.0000000e+02, -1.0000000e+02]],

       

In [ ]:
# compute accuracy
accuracy_metric = load_metric("accuracy")
accuracy_metric.compute(predictions=dataset_test_pred, references=test_dataset['label'])

/zhome/a6/6/127219/miniconda3/envs/sent_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


{'accuracy': 1.0}